### Data Visualization

In [1]:
import os

# Base directory where your data is located
base_path = r'/workspace/bdd100k/BDD100k/images'

# The subdirectories should be direct children of the 100k folder
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')
val_path = os.path.join(base_path, 'val')

# First, check if the directories exist
print(f"Train path exists: {os.path.exists(train_path)}")
print(f"Test path exists: {os.path.exists(test_path)}")
print(f"Val path exists: {os.path.exists(val_path)}")

# If the above shows False, let's see what directories actually exist in base_path
print("\nContents of base directory:")
print(os.listdir(base_path))

Train path exists: True
Test path exists: True
Val path exists: True

Contents of base directory:
['test', 'train', 'val']


In [21]:
import json

json_path = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k_labels_release\bdd100k\labels\bdd100k_labels_images_train.json'

with open(json_path, 'r') as f:
    labels_data = json.load(f)

num_labels = len(labels_data)

print(f"Number of labels in the JSON file: {num_labels}")

Number of labels in the JSON file: 69863


In [23]:
import os

base_train_path =  r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\train'

train_subdirs = ['testA', 'testB', 'trainA', 'trainB']

total_images = 0

num_base_train_images = len(os.listdir(base_train_path))
print(f"Number of images in {base_train_path}: {num_base_train_images}")
total_images += num_base_train_images


for subdir in train_subdirs:
    subdir_path = os.path.join(base_train_path, subdir)
    if os.path.isdir(subdir_path):
        num_images = len(os.listdir(subdir_path))
        print(f"Number of images in {subdir_path}: {num_images}")
        total_images += num_images
    else:
        print(f"Directory not found: {subdir_path}")

print(f"\nTotal number of images: {total_images}")

Number of images in C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\train: 1160
Number of images in C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\train\testA: 4130
Number of images in C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\train\testB: 2748
Number of images in C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\train\trainA: 37216
Number of images in C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\train\trainB: 24750

Total number of images: 70004


In [24]:
import os
import json

base_train_path =  r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\train'
json_path = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k_labels_release\bdd100k\labels\bdd100k_labels_images_train.json'

train_subdirs = ['testA', 'testB', 'trainA', 'trainB']

# Get list of all image filenames in the training directories and subdirectories
image_filenames = set()
for subdir in train_subdirs:
    subdir_path = os.path.join(base_train_path, subdir)
    if os.path.isdir(subdir_path):
        for filename in os.listdir(subdir_path):
            if filename.endswith('.jpg'): # Assuming images are JPGs
                image_filenames.add(filename)

# Add images from the base train path
for filename in os.listdir(base_train_path):
    if filename.endswith('.jpg'):
        image_filenames.add(filename)


# Load label data from JSON file
with open(json_path, 'r') as f:
    labels_data = json.load(f)

# Get list of filenames from label data
label_filenames = set([item['name'] for item in labels_data])

# Find the number of images that have corresponding labels
images_with_labels = image_filenames.intersection(label_filenames)

print(f"Total number of images in specified directories: {len(image_filenames)}")
print(f"Total number of entries in label JSON file: {len(label_filenames)}")
print(f"Number of images with corresponding labels: {len(images_with_labels)}")

Total number of images in specified directories: 70000
Total number of entries in label JSON file: 69863
Number of images with corresponding labels: 69863


## Task
Train a YOLOv8 model on the BDD100k dataset using the images in "/content/bdd100k/bdd100k/images/100k/train" and the labels in "/content/bdd100k_labels_release/bdd100k/labels/bdd100k_labels_images_train.json".

### Install necessary libraries

#### Subtask:
Install `ultralytics` which contains the YOLOv8 implementation and other dependencies.


**Reasoning**:
The subtask is to install the `ultralytics` package. I will use pip to install the package.



In [29]:
%pip install ultralytics

  Using cached torch-2.9.0-cp312-cp312-win_amd64.whl.metadata (30 kB)
Using cached torch-2.9.0-cp312-cp312-win_amd64.whl (109.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch None
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
error: uninstall-no-record-file

× Cannot uninstall torch None
╰─> The package's contents are unknown: no RECORD file was found for torch.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps torch==2.6.0+cu126


### Prepare the dataset

#### Subtask:
Convert the BDD100k labels to the YOLO format. This involves parsing the JSON file and creating text files for each image with bounding box information.


### **Reasoning**:
Convert the BDD100k labels to the YOLO format by parsing the JSON file and creating text files for each image with bounding box information. This involves defining paths, creating output directories, loading JSON data, iterating through images and annotations, converting bounding box coordinates to YOLO format, and writing the formatted data to text files. A category mapping is also needed.



In [4]:
import os, json, glob, re
from collections import Counter

# ===== paths =====
IMAGES_DIR  = r'C:\Users\anhhu\Downloads\Company Internship\BDD100k\images\train'
LABELS_JSON = r'C:\Users\anhhu\Downloads\Company Internship\BDD100k\labels\bdd100k_labels_release\bdd100k\labels\bdd100k_labels_images_train.json'
LABELS_ROOT = r'C:\Users\anhhu\Downloads\Company Internship\BDD100k\labels\train'

os.makedirs(LABELS_ROOT, exist_ok=True)

# ===== 10 lớp mục tiêu (thứ tự cố định) =====
CLS_MAP = {
    'person': 0,
    'rider': 1,
    'car': 2,
    'bus': 3,
    'truck': 4,
    'bike': 5,
    'motor': 6,
    'traffic light': 7,
    'traffic sign': 8,
    'train': 9,
}

# Các biến thể → chuẩn 10 lớp
NORMALIZE_TABLE = {
    'person': 'person',
    'rider': 'rider',
    'car': 'car',
    'bus': 'bus',
    'truck': 'truck',
    'bicycle': 'bike',
    'bike': 'bike',
    'motorcycle': 'motor',
    'motor': 'motor',
    'traffic light': 'traffic light',
    'traffic sign': 'traffic sign',
    # các biến thể hay gặp:
    'traffic-light': 'traffic light',
    'traffic_light': 'traffic light',
    'trafficlight': 'traffic light',
    'traffic-sign': 'traffic sign',
    'traffic_sign': 'traffic sign',
    'trafficsign': 'traffic sign',
}

def normalize_cat(raw: str) -> str | None:
    if not raw:
        return None
    # ép utf-8 safe, lower, strip
    s = raw.encode('utf-8', 'ignore').decode('utf-8').lower()
    # thay \xa0, tab, nhiều space về 1 space
    s = s.replace('\xa0', ' ')
    s = re.sub(r'[\t\r\n]+', ' ', s)
    # unify các dấu nối
    s = s.replace('-', ' ').replace('_', ' ')
    s = re.sub(r'\s+', ' ', s).strip()
    # tra bảng biến thể trước
    if s in NORMALIZE_TABLE:
        s = NORMALIZE_TABLE[s]
    # cuối cùng: nếu đã khớp CLS_MAP thì nhận
    return s if s in CLS_MAP else None

W, H = 1280.0, 720.0  # BDD100K mặc định

# Index ảnh theo basename -> relative path
image_index = {}
for p in glob.glob(os.path.join(IMAGES_DIR, '**', '*.jpg'), recursive=True):
    rel = os.path.relpath(p, IMAGES_DIR).replace('\\', '/')
    image_index[os.path.basename(p)] = rel

with open(LABELS_JSON, 'r', encoding='utf-8') as f:
    items = json.load(f)

miss_img = 0
unk_cls = 0
files_written = 0
boxes_written = 0
unknown_examples = Counter()

for it in items:
    name = it.get('name')
    rel_img = image_index.get(name)
    if rel_img is None:
        miss_img += 1
        continue

    rel_lbl  = os.path.splitext(rel_img)[0] + '.txt'
    out_path = os.path.join(LABELS_ROOT, rel_lbl)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    labels = it.get('labels', [])
    lines = []
    for lb in labels:
        box = lb.get('box2d')
        raw = lb.get('category', '')

        # 1) Nếu KHÔNG có box2d (vd lane, drivable area = poly2d) → bỏ qua hẳn
        if not box:
            continue

        # 2) Có box2d ⇒ mới normalize & map
        cat = normalize_cat(raw)
        if not cat:
            # chỉ cộng unknown nếu annotation có box2d nhưng cat không map
            unknown_examples[raw] += 1
            unk_cls += 1
            continue

        x1, x2 = float(min(box['x1'], box['x2'])), float(max(box['x1'], box['x2']))
        y1, y2 = float(min(box['y1'], box['y2'])), float(max(box['y1'], box['y2']))

        # clip biên
        x1 = max(0.0, min(x1, W)); x2 = max(0.0, min(x2, W))
        y1 = max(0.0, min(y1, H)); y2 = max(0.0, min(y2, H))
        bw, bh = x2 - x1, y2 - y1
        if bw <= 1e-6 or bh <= 1e-6:
            continue

        cx = (x1 + x2) / 2.0 / W
        cy = (y1 + y2) / 2.0 / H
        bw /= W; bh /= H

        lines.append(f"{CLS_MAP[cat]} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    if lines:
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(lines) + '\n')
        files_written += 1
        boxes_written += len(lines)
    else:
        # xoá file rỗng cũ nếu có
        if os.path.exists(out_path) and os.path.getsize(out_path) == 0:
            try: os.remove(out_path)
            except: pass

print("DONE convert.")
print("Label files written:", files_written, " total boxes:", boxes_written)
print("Missing images in disk (but in JSON):", miss_img)
print("Unknown-class annotations skipped:", unk_cls)

# In top 15 nhãn lạ (nếu còn)
if unknown_examples:
    print("\nTop unknown categories (raw) → cần bổ sung NORMALIZE nếu hợp lệ:")
    for k, v in unknown_examples.most_common(15):
        print(f"- {repr(k)}: {v}")


DONE convert.
Label files written: 69863  total boxes: 1286871
Missing images in disk (but in JSON): 0
Unknown-class annotations skipped: 0


In [5]:
import os, glob
LB_TRAIN = r"C:\Users\anhhu\Downloads\Company Internship\BDD100k\labels\train"
txts = glob.glob(os.path.join(LB_TRAIN, "**", "*.txt"), recursive=True)
non_empty = sum(1 for p in txts if os.path.getsize(p) > 0)
print("txt files:", len(txts), " | non-empty:", non_empty)


txt files: 69863  | non-empty: 69863


##Kiểm tra dữ liệu


In [53]:
import os, glob

IM_TRAIN = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\train'
LB_TRAIN = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\train'

# Đếm ảnh & nhãn hiện có
imgs = glob.glob(os.path.join(IM_TRAIN, "**", "*.jpg"), recursive=True)
txts = glob.glob(os.path.join(LB_TRAIN, "**", "*.txt"), recursive=True)
print("Images train:", len(imgs))          # ~70,000
print("Label files train:", len(txts))     # ~69,863 nếu chưa tạo rỗng

# (Tuỳ chọn) tạo .txt rỗng để đủ 70,000 ảnh
need_empty = len(imgs) - len(txts)
if need_empty > 0:
    made = 0
    for img in imgs:
        rel = os.path.relpath(img, IM_TRAIN).replace("\\","/")
        txt = os.path.join(LB_TRAIN, os.path.splitext(rel)[0] + ".txt")
        if not os.path.exists(txt):
            os.makedirs(os.path.dirname(txt), exist_ok=True)
            open(txt, "w").close()
            made += 1
    print("Empty label files created:", made)

# Kiểm tra lần nữa
txts = glob.glob(os.path.join(LB_TRAIN, "**", "*.txt"), recursive=True)
print("Label files train (after):", len(txts))


Images train: 70000
Label files train: 70000
Label files train (after): 70000


In [ ]:
#Conver json to txt for Yolo
import os, json, glob

IMAGES_DIR = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\images\100k\val'
LABELS_JSON = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k_labels_release\bdd100k\labels\bdd100k_labels_images_val.json'
LABELS_ROOT = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\labels\val'

os.makedirs(LABELS_ROOT, exist_ok=True)

CLS_MAP = {'person':0,'rider':1,'car':2,'bus':3,'truck':4,'bike':5,'motor':6,'traffic light':7,'traffic sign':8,'train':9}
W, H = 1280.0, 720.0

# index ảnh: basename -> relative path (soi gương)
image_index = {}
for p in glob.glob(os.path.join(IMAGES_DIR, '**', '*.jpg'), recursive=True):
    rel = os.path.relpath(p, IMAGES_DIR).replace('\\','/')
    image_index[os.path.basename(p)] = rel

with open(LABELS_JSON, 'r') as f:
    items = json.load(f)

files_written = miss_img = 0
for it in items:
    name = it.get('name')
    rel_img = image_index.get(name)
    if rel_img is None:
        miss_img += 1
        continue

    rel_lbl = os.path.splitext(rel_img)[0] + '.txt'
    out_path = os.path.join(LABELS_ROOT, rel_lbl)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    lines = []
    for lb in it.get('labels', []):
        if 'box2d' not in lb or 'category' not in lb: continue
        cat = lb['category']
        if cat not in CLS_MAP: continue
        b = lb['box2d']
        x1, x2 = float(min(b['x1'], b['x2'])), float(max(b['x1'], b['x2']))
        y1, y2 = float(min(b['y1'], b['y2'])), float(max(b['y1'], b['y2']))
        # clip
        x1=max(0,min(x1,W)); x2=max(0,min(x2,W))
        y1=max(0,min(y1,H)); y2=max(0,min(y2,H))
        bw, bh = x2-x1, y2-y1
        if bw<=1e-6 or bh<=1e-6: continue
        cx=(x1+x2)/2.0/W; cy=(y1+y2)/2.0/H; bw/=W; bh/=H
        lines.append(f"{CLS_MAP[cat]} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    if lines:
        with open(out_path,'w') as f: f.write('\n'.join(lines))
        files_written += 1

print("VAL convert:", files_written, "files written; missing images in JSON:", miss_img)


VAL convert: 10000 files written; missing images in JSON: 0


In [51]:
import os, glob

IM_VAL = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\val'
LB_VAL = r"C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\val"

imgs = glob.glob(os.path.join(IM_VAL, "**", "*.jpg"), recursive=True)
txts = glob.glob(os.path.join(LB_VAL, "**", "*.txt"), recursive=True)
non_empty = [t for t in txts if os.path.getsize(t) > 0]

print("VAL images:", len(imgs))          # kỳ vọng ~10000
print("VAL label files:", len(txts))     # kỳ vọng ~10000 (hoặc ~số ảnh có bbox)
print("VAL non-empty labels:", len(non_empty))  # PHẢI >0


# Lấy 1 ảnh bất kỳ và kiểm tra file nhãn “soi gương”
sample_img = imgs[0]
rel = os.path.relpath(sample_img, IM_VAL).replace("\\","/")
sample_txt = os.path.join(LB_VAL, os.path.splitext(rel)[0] + ".txt")
print("Sample img:", sample_img)
print("Expect txt:", sample_txt, " exists:", os.path.exists(sample_txt),
      " size:", os.path.getsize(sample_txt) if os.path.exists(sample_txt) else -1)


VAL images: 10000
VAL label files: 10000
VAL non-empty labels: 10000
Sample img: C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\val\b1c66a42-6f7d68ca.jpg
Expect txt: C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\val\b1c66a42-6f7d68ca.txt  exists: True  size: 1324


###Xóa cache và hướng đúng đường dẫn

### Dọn dữ liệu trùng

In [49]:
import os, glob, shutil

LB_ROOT   = r'C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\train'
NESTED_TR = os.path.join(LB_ROOT, 'train')  # nhánh dư thường gặp

if os.path.isdir(NESTED_TR):
    moved, replaced, removed = 0, 0, 0
    for src in glob.glob(os.path.join(NESTED_TR, '**', '*.txt'), recursive=True):
        rel = os.path.relpath(src, NESTED_TR)         # đường dẫn tương đối TỪ nhánh dư
        dst = os.path.join(LB_ROOT, rel)              # đích mong muốn ở LB_ROOT
        os.makedirs(os.path.dirname(dst), exist_ok=True)

        if os.path.exists(dst):
            # Nếu file đích rỗng và src có nội dung -> thay thế
            if os.path.getsize(dst) == 0 and os.path.getsize(src) > 0:
                shutil.move(src, dst)
                replaced += 1
            else:
                # Ngược lại giữ dst, xóa src dư
                os.remove(src)
                removed += 1
        else:
            shutil.move(src, dst)
            moved += 1

    # dọn thư mục rỗng
    for root, dirs, files in os.walk(NESTED_TR, topdown=False):
        for f in files:
            pass
        for d in dirs:
            p = os.path.join(root, d)
            if not os.listdir(p):
                os.rmdir(p)
    try:
        os.rmdir(NESTED_TR)
    except OSError:
        pass

    print(f'Done cleanup. moved={moved}, replaced={replaced}, removed_dups={removed}')
else:
    print('Không thấy nhánh dư:', NESTED_TR)

# Đếm lại để xác nhận
import glob
total = len(glob.glob(os.path.join(LB_ROOT, '**', '*.txt'), recursive=True))
print('train txt after cleanup:', total)


Không thấy nhánh dư: C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\train\train
train txt after cleanup: 70000


In [ ]:
import glob
print("train txt:", len(glob.glob(r"C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\train\**\*.txt", recursive=True))) 
print("val   txt:", len(glob.glob(r"C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\val**\*.txt",   recursive=True)))


train txt: 70000
val   txt: 10000


In [58]:
import os, glob

train_txts = glob.glob(r"C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\train\**\*.txt", recursive=True)
val_txts   = glob.glob(r"C:\Users\anhhu\Downloads\Company Internship\archive\bdd100k\bdd100k\100k\labels\val**\*.txt",   recursive=True)  

train_non_empty = sum(1 for p in train_txts if os.path.getsize(p) > 0)
val_non_empty   = sum(1 for p in val_txts   if os.path.getsize(p) > 0)

print("Train non-empty labels:", train_non_empty)
print("Val   non-empty labels:", val_non_empty)


Train non-empty labels: 0
Val   non-empty labels: 10000


###Xóa cache

In [ ]:
import os
for p in [
    '/content/bdd100k/bdd100k/images/100k/train.cache',
    '/content/bdd100k/bdd100k/labels/100k/train.cache',
    '/content/bdd100k/bdd100k/images/100k/val.cache',
    '/content/bdd100k/bdd100k/labels/100k/val.cache',
]:
    if os.path.exists(p):
        os.remove(p)
print('Cache cleared.')


Cache cleared.


###YOLO v8 Running

In [ ]:
import os, yaml

# === 1) Khai báo đúng 10 lớp (theo mapping bạn đã convert) ===
class_names = [
    'person', 'rider', 'car', 'bus', 'truck',
    'bike', 'motor', 'traffic light', 'traffic sign', 'train'
]

# === 2) Tạo data.yaml đúng với cây thư mục hiện tại ===
ROOT = r"C:\Users\anhhu\Downloads\Company Internship\BDD100k"   # thư mục chứa images/ và labels/
yaml_file = r"C:\Users\anhhu\Downloads\Company Internship\bdd100k.yaml"

data = {
    "path": ROOT.replace("\\", "/"),   # dùng / cho chắc ăn trên Windows
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",             # có cũng được, không có YOLO sẽ bỏ qua
    "names": class_names
}

os.makedirs(os.path.dirname(yaml_file), exist_ok=True)
with open(yaml_file, "w", encoding="utf-8") as f:
    yaml.dump(data, f, sort_keys=False, allow_unicode=True)

print(f"YOLO config saved to: {yaml_file}")

# (khuyến nghị) sanity check nhanh để chắc thư mục tồn tại
for p in [ROOT, os.path.join(ROOT, "images/train"), os.path.join(ROOT, "images/val")]:
    print("OK" if os.path.exists(p) else "MISSING", "->", p)


YOLO config saved to: C:\Users\anhhu\Downloads\Company Internship\bdd100k.yaml
OK -> C:\Users\anhhu\Downloads\Company Internship\BDD100k
OK -> C:\Users\anhhu\Downloads\Company Internship\BDD100k\images/train
OK -> C:\Users\anhhu\Downloads\Company Internship\BDD100k\images/val


In [2]:
from ultralytics import YOLO
import os, yaml
import torch

# === 0) Đường dẫn file YAML đã tạo ở bước trước ===
yaml_file = r"C:\Users\anhhu\Downloads\Company Internship\bdd100k.yaml"
assert os.path.exists(yaml_file), f"Không thấy YAML: {yaml_file}"

# === 1) Chọn weights nhất quán (v8 hoặc v11, đừng trộn) ===
# WEIGHTS = "yolov11n.pt"   # nếu bạn dùng YOLOv11
WEIGHTS = "yolov8n.pt"       # bạn đang dùng v8n → giữ nguyên

# === 2) Khởi tạo model ===
model = YOLO(WEIGHTS)
model.info()

# === 3) Thiết lập thiết bị và workers cho Windows ===
# - Windows đôi khi bị lỗi DataLoader khi workers quá cao → đặt 0–2 cho chắc
# - device=0 nếu có GPU CUDA, ngược lại chuyển "cpu"
device_arg = 0 if torch.cuda.is_available() else "cpu"
num_workers = 2  # nếu vẫn lỗi DataLoader, hạ xuống 0

# === 4) Train ===
results = model.train(
    data=yaml_file,     # đường dẫn file YAML
    epochs=5,           # thử nhanh; train thật dùng 50–100
    imgsz=640,          # train thật có thể đẩy 960–1280
    batch=-1,           # auto batch theo VRAM
    workers=num_workers,
    device=device_arg,
    deterministic=True, # tái lập kết quả tốt hơn trên Windows
    cache=True,         # cache ảnh để tăng tốc (nếu đủ RAM/đĩa)
    close_mosaic=10,    # setting mặc định hợp lý cho v8
    pretrained=True     # dùng backbone tiền huấn luyện
)

# === 5) In thư mục lưu kết quả ===
try:
    print("Save dir:", results.save_dir)
except Exception:
    try:
        print("Save dir:", model.trainer.save_dir)
    except Exception:
        pass


YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
Ultralytics 8.3.221  Python-3.12.3 torch-2.9.0+cpu CPU (AMD Ryzen 5 4600H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\anhhu\Downloads\Company Internship\bdd100k.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False,

KeyboardInterrupt: 

### Đánh giá kết quả